In [1]:
import pandas as pd
import os
import sys

In [2]:
# Set paths
homey = os.getcwd()
redouxPath = os.path.join(homey, 'ForecastRedoux')
sqlPath = os.path.join(redouxPath, 'SQL')
rawDataPath = os.path.join(redouxPath, 'RawData')
AdditionalInfoPath = os.path.join(homey, 'AdditionalInfo')

In [3]:
# Pull in most recent timeline
timeline = pd.read_excel(os.path.join(homey, 'RegularForecast.xlsx'), sheetname='Timeline')

In [4]:
# Connect to FB API
sys.path.insert(0, 'Z:\Python projects\FishbowlAPITestProject')
import connecttest

In [5]:
# Query part costs
myresults = connecttest.create_connection(sqlPath, 'AvgCostQuery.txt')
myexcel = connecttest.makeexcelsheet(myresults)
connecttest.save_workbook(myexcel, rawDataPath, 'AvgCosts.xlsx')

Success!


In [6]:
# Get part costs into a dataFrame
costpath = os.path.join(rawDataPath, 'AvgCosts.xlsx')
avgCost = pd.read_excel(costpath) #Opens and puts the data into a dataframe
avgCost = avgCost.sort_values(by='PART', ascending=True) #Sort the data by part

In [7]:
# Merge part costs onto timeline
timeline = pd.merge(timeline.copy(), avgCost.copy(), how='left', on='PART')

In [8]:
# Calculate total costs
timeline['TotalCost'] = timeline['QTYREMAINING'] * timeline['AvgCost']

In [9]:
# Grab list of imaginary builds and create build number reference
newbuilds = pd.read_excel(os.path.join(AdditionalInfoPath, 'PartsToBuild.xlsx'), sheetname='Sheet1')
newbuilds.reset_index(inplace=True)
newbuilds['buildIndex'] = (newbuilds['index'] * -1) -1
newbuilds.drop('index', axis=1, inplace=True)

In [10]:
# Create columns to be filled by the next loop.
newbuilds['Total_Purchase'] = 0
newbuilds['Total_Cost_Produced'] = 0

In [27]:
# This dataFrame will hold cost issues for all the imaginary builds
missingCost = pd.DataFrame()

In [28]:
""" This loop calculates and updates the total purchasing need for each imaginary build
    and it also finds the total raw material cost. """
for index, row in newbuilds.iterrows():
    imagBuild = row['buildIndex']
    thisBuild = timeline[timeline['GRANDPARENT'] == imagBuild].copy()
    purchDF = thisBuild[thisBuild['ORDERTYPE'] == 'Purchase'].copy()
    totPurchDF = purchDF[purchDF['TotalCost'].notnull()].copy()
    newbuilds.set_value(index, 'Total_Purchase', purchDF['TotalCost'].sum())
    # Now adding the total raw good cost of "Buy" items to show total amount in movement
    buyItemsDF = thisBuild[thisBuild['Make/Buy'] == 'Buy'].copy()
    rawGoodsDF = buyItemsDF[buyItemsDF['ORDERTYPE'] == 'Raw Good']
    newbuilds.set_value(index, 'Total_Cost_Produced', rawGoodsDF['TotalCost'].sum())
    # missingCost records parts that have 0 cost or are completely missing cost
    noCostDF = rawGoodsDF[(rawGoodsDF['TotalCost'].isnull()) | (rawGoodsDF['TotalCost'] == 0)].copy() # ... it's so nice ...
    noCostDF['Build'] = newbuilds.get_value(index, 'Part')
    missingCost = missingCost.copy().append(noCostDF.copy())

In [32]:
# Create a column representing cost in material either in inventory currently or already on order
newbuilds['Current_Cost_Used'] = - (newbuilds['Total_Purchase'] + newbuilds['Total_Cost_Produced'])

In [29]:
missingCost.to_clipboard()

In [13]:
newbuilds.to_clipboard()

In [14]:
timeline.to_clipboard()

In [33]:
newbuilds

,Part,Qty,Date,buildIndex,Total_Purchase,Total_Cost_Produced,Current_Cost_Used
0,016-1632-10 r01,8,2017-05-15,-1,118,-3985,3867
1,016-452-10 r03,54,2017-05-15,-2,2411,-17473,15062
2,016-1622-10 r02,35,2017-05-15,-3,806,-11976,11170
3,016-1922-10 r02,40,2017-05-15,-4,145304,-194230,48926
4,016-1009-10 r02,40,2017-05-15,-5,29202,-57554,28352
5,016-1877-10 r03,370,2017-05-15,-6,176836,-196521,19685
6,016-1847-10 r01,10,2017-05-15,-7,4442,-6386,1944
7,016-1849-10 r01,10,2017-05-15,-8,2362,-5568,3206
8,016-1875-10 r01,30,2017-05-15,-9,8525,-14561,6036
9,016-550-10 r01,10,2017-05-15,-10,9821,-16129,6308
